# Medication Extraction

This notebook is intended as an example of extracting medications and their attributes from text.
You can specify your own dataset (must contain text), sampler, schema, model, prompting strategy and templates or use the default ones.

In [ ]:
import sys
import os
sys.path.append(os.getcwd()+"/../..")
from src import paths

import os

import torch

from src import paths
from src.utils import (load_model_and_tokenizer,   
                       check_gpu_memory,
                       get_format_fun,
                        information_retrieval, 
                        get_sampler,
                        get_outlines_generator,
                        get_pydantic_schema,
                        format_prompt,
                        outlines_medication_prompting,
)

import argparse

from datasets import Dataset

import json

import random

In [ ]:
MODEL_NAME = "Llama2-MedTuned-13b"
QUANTIZATION = "4bit"
PROMPT_STRATEGY = "few_shot_instruction"
SAMPLER_NAME = "greedy"
ATTN_IMPLEMENTATION = "flash_attention_2" # Only enable this if your GPU is able to use flash_attention_2, check https://github.com/Dao-AILab/flash-attention
MAX_TOKENS = 500
NUM_EXAMPLES = 10

In [ ]:
# Check GPU Memory, for 13B model should have at least 16GB of VRAM (12 could work for short examples)
check_gpu_memory()

In [ ]:
# Load Model and Tokenizer
model, tokenizer = load_model_and_tokenizer(model_name = MODEL_NAME,
                                            task_type = "outlines",
                                            quantization = QUANTIZATION,
                                            attn_implementation = ATTN_IMPLEMENTATION,
                                            )
check_gpu_memory()

In [ ]:
# Output schema (you can implement your own, must follow pydantic BaseModel, or check https://github.com/outlines-dev/outlines
schema = get_pydantic_schema(schema_name="medication")

# Example of what this looks like
get_default_pydantic_model("medication")

In [ ]:
# Get sampler (could also be "multionmial" or "beam")
sampler = get_sampler(SAMPLER_NAME)

# Get Generator (for task with JSON output according to schema)
generator = get_outlines_generator(model, sampler, task = "json", schema = schema)

In [ ]:
# Load your data
df = Dataset.load_from_disk(paths.DATA_PATH_PREPROCESSED/"medication/kisim_medication_sample")
text = df["text"][1] # Sample of 1

In [ ]:
# Write your task instruction, system prompt and examples or load default ones
with open(paths.DATA_PATH_PREPROCESSED/"medication/task_instruction.txt", "r") as f:
    task_instruction = f.read()

with open(paths.DATA_PATH_PREPROCESSED/"medication/system_prompt.txt", "r") as f:
    system_prompt = f.read()

with open(paths.DATA_PATH_PREPROCESSED/"medication/examples.json", "r") as file:
    examples = json.load(file)                  

# Can also select a subset of examples at random
random.seed(42)
examples = random.sample(examples, NUM_EXAMPLES)

In [ ]:
# Load a prompting template and format input
format_fun = get_format_fun(prompting_strategy=PROMPT_STRATEGY)
input = format_prompt(text = text, format_fun=format_fun, task_instruction = task_instruction, system_prompt = system_prompt, examples = examples)

# If you don't want to use any of the templates and formatting you can also use the tokenizer to prepare an input:
# input = tokenizer(["This is a test"], return_tensors = "pt")

In [ ]:
input

In [ ]:
# Get results (note you can also use DataLoaders to submit
result = generator(input, max_tokens = MAX_TOKENS)

In [ ]:
# Alternatively you can stream the input and observe the progress
for token in generator.stream(input, max_tokens = MAX_TOKENS):
    print(token, end = "")